# Dimensionality Reduction

In [1]:
from src.preprocessing import load_data
from pathlib import Path

In [2]:
data_path = Path('data')
file_path = data_path / '23MAG_HW_1_v1.xls'

In [3]:
load_data(file_path)

,Age,Number of dependents,Salary,Years of stay in the region,Car price,Amount of credit
0,38.0,2.0,22600.0,8.0,22000.0,NaN
1,47.0,2.0,11200.0,14.0,12000.0,119000.0
2,NaN,2.0,22500.0,9.0,22000.0,224000.0
3,51.0,3.0,16000.0,17.0,18000.0,173000.0
4,41.0,2.0,NaN,11.0,6000.0,57000.0
...,...,...,...,...,...,...
67,39.0,2.0,10900.0,10.0,11000.0,107000.0
68,20.0,0.0,9900.0,0.0,NaN,72000.0
69,NaN,1.0,20700.0,3.0,18000.0,193000.0
70,39.0,2.0,23700.0,9.0,24000.0,237000.0
